In [55]:
# Do not delete this cell. It ensures that you can do the imports,
# load datasets etc. in the same fashion as in any Python script
# in the project template.


import sys
sys.path.insert(0, '../..')
from bld.project_paths import project_paths_join as ppj
from bld.project_paths import project_paths as pp



In [56]:
import numpy as np
import json
import pickle
import pandas as pd
from scipy.stats import sem
import seaborn as sns
import matplotlib.pyplot as plt
%config Completer.use_jedi = False

from matplotlib import rc
rc('text', usetex=True)
plt.rcParams.update({'font.size': 20})


# Are fully algortihmic markets more collusive than fully human markets?

To answer this question we will compare the outcomes for the algorithmic market to the outcomes in the last super game of the human markets. We focus on the last super game as this allowed humans for some learning. Arguably this is the most accurate comparision.

First we load the needed data.

In [57]:
# Two firm algorithm markets
with open(ppj("OUT_DATA", f"grid_2_agents.pickle"), "rb") as f:
    all_output_grids_2_agents = pickle.load(f)
all_prices_2_agents_grid = np.array(all_output_grids_2_agents['avg_price'])
with open(ppj("OUT_DATA", "super_star_avg_prices_2_agents.pickle"), "rb") as f:
    super_star_avg_prices_2_agents = pickle.load(f)

# Three firm algorithm markets
with open(ppj("OUT_DATA", f"grid_3_agents.pickle"), "rb") as f:
    all_output_grids_3_agents = pickle.load(f)
all_prices_3_agents_grid = np.array(all_output_grids_3_agents['avg_price'])
with open(ppj("OUT_DATA", "super_star_avg_prices_3_agents.pickle"), "rb") as f:
    super_star_avg_prices_3_agents = pickle.load(f)


In [58]:
# we want to transform those to pandas dataframes as it will be easier to deal with them once we go over to plotting

In [59]:
with open(ppj("OUT_DATA", "data_group_level.pickle"), "rb") as f:
    data_group_level = pickle.load(f)

## Next up we use mann whitney u tests to compare the respective samples.

In [60]:
from scipy.stats import mannwhitneyu, ttest_1samp

### We start by comparing the market prices within a respective market size.

Two firm market:

In [61]:
print("Two firm market. Algo (super star) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                                        (data_group_level['treatment'] == '2H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = mannwhitneyu(
        subset_sg['winning_price'],
        super_star_avg_prices_2_agents,
        use_continuity=False,
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Two firm market. Algo (super star) v human 
Super game: 1. P-value: 0.000
Super game: 2. P-value: 0.000
Super game: 3. P-value: 0.000


In [62]:
print("Two firm market. Algo (grid average) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                      (data_group_level['treatment'] == '2H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = ttest_1samp(
        subset_sg['winning_price'],
        all_prices_2_agents_grid.mean(),
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Two firm market. Algo (grid average) v human 
Super game: 1. P-value: 0.000
Super game: 2. P-value: 0.005
Super game: 3. P-value: 0.034


In [63]:
print(f"Average price of the grid in two firm markets: {all_prices_2_agents_grid.mean():.3f}")

Average price of the grid in two firm markets: 3.511


Three firm market:


In [64]:
print("Three firm market. Algo (super star) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                     (data_group_level['treatment'] == '3H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = mannwhitneyu(
        subset_sg['winning_price'],
        super_star_avg_prices_3_agents,
        use_continuity=False,
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Three firm market. Algo (super star) v human 
Super game: 1. P-value: 0.000
Super game: 2. P-value: 0.008
Super game: 3. P-value: 0.980


In [65]:
print("Two firm market. Algo (grid average) v human ")
for sg in range(1,4):
    subset_sg = data_group_level.loc[(data_group_level['super_game'] == sg) & 
                                      (data_group_level['treatment'] == '3H0A')].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()

    p_value = ttest_1samp(
        subset_sg['winning_price'],
        all_prices_3_agents_grid.mean(),
        alternative='two-sided'
    )[-1]
    print(f"Super game: {sg}. P-value: {p_value:.3f}")

Two firm market. Algo (grid average) v human 
Super game: 1. P-value: 0.991
Super game: 2. P-value: 0.339
Super game: 3. P-value: 0.044


In [66]:
print(f"Average price of the grid in three firm markets: {all_prices_3_agents_grid.mean():.3f}")

Average price of the grid in three firm markets: 1.574


There exist no statistically significant differences between the prices in the three firm markets with the super star algorithms and humans. Interestingly, the average algorithm from the grid search is *less* collusive than the human market with this difference being statistically significant at $p<0.01$.

# Algo v Algo

See Algo analysis q_env.ipynb


# Human v humans


In [67]:
data_humans_first_super_game = data_group_level.loc[(data_group_level['super_game'] == 1) & 
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()
data_humans_second_super_game = data_group_level.loc[(data_group_level['super_game'] == 2) & 
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()
data_humans_third_super_game = data_group_level.loc[(data_group_level['super_game'] == 3) & 
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()


In [68]:
data_humans_all_super_game = data_group_level.loc[
                                                    (data_group_level['treatment'].isin(['3H0A', '2H0A']))].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()


In [69]:
mannwhitneyu(
    data_humans_first_super_game.loc[data_humans_first_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_first_super_game.loc[data_humans_first_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.04530386472278079

In [70]:
mannwhitneyu(
    data_humans_second_super_game.loc[data_humans_second_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_second_super_game.loc[data_humans_second_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.050961936967763306

In [71]:
mannwhitneyu(
    data_humans_third_super_game.loc[data_humans_third_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_third_super_game.loc[data_humans_third_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.28275614744399247

In [72]:
mannwhitneyu(
    data_humans_all_super_game.loc[data_humans_all_super_game['treatment'] == '3H0A']['winning_price'],
    data_humans_all_super_game.loc[data_humans_all_super_game['treatment'] == '2H0A']['winning_price'],
    use_continuity=False,
    alternative='two-sided'

)[-1]

0.04042397933690852

This shows the importance of learning for humans, which appears to be more pronounce in larger and therefore strategically more complex markets.

# Mixed market analysis

#### Different states

There are states that trigger responses by the algorithm that are different from the ones discussed in the paper. Below you see that those can account for only 1% of all rounds.

In [73]:
data_algo = data_group_level.dropna(subset=['price_algorithm']).copy()

In [74]:
len(data_algo.loc[data_algo['price_algorithm'] == 2]) +  len(data_algo.loc[data_algo['price_algorithm'] == 3]) 

49

In [75]:
49 / len(data_algo)

0.012493625701172871

#### Analysis

In [78]:
data_mixed_pooled = data_group_level.loc[(data_group_level['treatment'].isin(['3H0A', '2H0A', '1H1A', '2H1A', '1H2A'])
                                                    )].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price', 'collusive']].mean()


2H0A v 1H1A (pooled)

In [85]:
mannwhitneyu(
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '2H0A']['winning_price'],
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '1H1A']['winning_price'],
use_continuity=False,
alternative='two-sided'

)[-1]


0.83563115352624

3H0A v 2H1A

In [90]:
mannwhitneyu(
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '3H0A']['winning_price'],
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '2H1A']['winning_price'],
use_continuity=False,
alternative='two-sided'

)[-1]


0.06391934147515746

2H1A v 1H2A

In [92]:
mannwhitneyu(
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '2H1A']['winning_price'],
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '1H2A']['winning_price'],
use_continuity=False,
alternative='two-sided'

)[-1]


0.005276795014053497

3H0A v 1H2A

In [93]:
mannwhitneyu(
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '3H0A']['winning_price'],
data_mixed_pooled.loc[data_mixed_pooled['treatment'] == '1H2A']['winning_price'],
use_continuity=False,
alternative='two-sided'

)[-1]


0.17542896105941352